# Comparison of GPForecasting GaussianProcersses and GPy

The notebook includes a profound comparison between 3 Gaussian process implementations:
- `GPForecasting.jl`
- `GaussianProcess.jl`
- `GPy`

The tests for comparison here are used in the unit test of `synthetic.jl` of `GPForecasting.jl`.
Comparisons are perfromed for:
- one dimensional input
- multidimensional input
- multidimensional output (`GPForecasting.jl` and `GPy` only)

The following metrics were used for comparison before and after hyperparameter optimization:
- marginal log likelihood (on the training data): `MLL_cov(train)`
- mean squared error on test data: `MSE(test)`
- mean log loss on test data using predicted mean and noiseless variance: `MLL_var(test)`
- mean log loss on test data using predicted mean and covariance: `MLL_cov(test)`

Notes:
- for `GaussianProcesses.jl` hyperparameter optimizations we used the same specifcations of omptimizations as in `GPForecasting.jl` as it's more stable
- results of multidimensional output problems can differ significantly for `GPForecasting.jl` and `GPy` due to the different implementations. These tests are therefore only indicative.

In [1]:
using Pkg
using GPForecasting
using GaussianProcesses
using PyCall
using PyPlot
using Distributions
using BenchmarkTools
using Conda
using LinearAlgebra
using Statistics
using Random: seed!
using Optim
using LineSearches
using FillArrays
@pyimport GPy as GPy

┌ Warning: `getindex(o::PyObject, s::AbstractString)` is deprecated in favor of dot overloading (`getproperty`) so elements should now be accessed as e.g. `o."s"` instead of `o["s"]`.
│   caller = top-level scope at none:0
└ @ Core none:0
┌ Warning: `@pyimport foo` is deprecated in favor of `foo = pyimport("foo")`.
│   caller = _pywrap_pyimport(::PyObject) at PyCall.jl:407
└ @ PyCall /Users/molet/.julia/packages/PyCall/a5Jd3/src/PyCall.jl:407


## Check versions

In [2]:
Pkg.status("GPForecasting")

    Status `~/.julia/environments/v1.1/Project.toml`
  [13ebecde] GPForecasting v1.0.2


In [3]:
Pkg.status("GaussianProcesses")

    Status `~/.julia/environments/v1.1/Project.toml`
  [891a1506] GaussianProcesses v0.9.0


In [4]:
GPy.__version__

"1.9.6"

## Optimization functions for GaussianProcesses and GPy using the same specifications as in GPForecasting

In [5]:
function learn!(gp::GPBase; its=200, trace=true, domean=true, kern=true, noise=true, lik=true)
    
    params_kwargs = GaussianProcesses.get_params_kwargs(gp; domean=domean, kern=kern, noise=noise, lik=lik)
    
    function f(hyp::Vector{Float64})
        GaussianProcesses.set_params!(gp, hyp; params_kwargs...)
        GaussianProcesses.update_target!(gp)
        return -gp.target
    end
    
    function g!(storage::Vector, hyp::Vector{Float64})
        GaussianProcesses.set_params!(gp, hyp; params_kwargs...)
        GaussianProcesses.update_target_and_dtarget!(gp; params_kwargs...)
        storage[:] = -gp.dtarget
    end
    
    params = GaussianProcesses.get_params(gp; params_kwargs...)
            
    results = optimize(
        f, 
        g!, 
        params, 
        LBFGS(alphaguess = LineSearches.InitialStatic(scaled=true), linesearch = LineSearches.BackTracking()),
        Optim.Options(
            x_tol = 1.0e-8,
            f_tol = 1.0e-8,
            g_tol = 1.0e-5,
            f_calls_limit = its,
            g_calls_limit = its,
            iterations = its,
            show_trace = trace,
        ),
    )
    
    return results
end

function learn!(gp; its=200, trace=true)
    
    params_names = gp.parameter_names()
    
    function f(hyp::Vector{Float64})
        for i = 1:length(hyp)
            gp.__setitem__(params_names[i], hyp[i])
        end
        gp.parameters_changed()
        return gp.objective_function()
    end
    
    function g!(storage::Vector, hyp::Vector{Float64})
        for i = 1:length(hyp)
            gp.__setitem__(params_names[i], hyp[i])
        end
        gp.parameters_changed()
        storage[:] = gp.objective_function_gradients()
    end
    
    params = gp.param_array
        
    results = optimize(
        f, 
        g!, 
        params, 
        LBFGS(alphaguess = LineSearches.InitialStatic(scaled=true), linesearch = LineSearches.BackTracking()),
        Optim.Options(
            x_tol = 1.0e-8,
            f_tol = 1.0e-8,
            g_tol = 1.0e-5,
            f_calls_limit = its,
            g_calls_limit = its,
            iterations = its,
            show_trace = trace,
        ),
    )
    
    return results
end
;

learn! (generic function with 2 methods)

## Loss functions

In [6]:
function mse(means, y_true)                                                                  
    return mean((y_true .- means).^2)                                                        
end

function mll_var(means, vars, y_true)
    return 0.5 * mean(log.(2π .* vars)) .+ 0.5 * mean((y_true .- means).^2 ./ vars)
end

function mll_cov(means, cov, y_true)
    return -logpdf(MvNormal(vec(means), cov), vec(y_true))
end
;

## Wrapper functions for 1 dimensional output tests

In [7]:
function gpforecasting(m, k, x_train, y_train, x_test, y_test; print_result=true, opt=true)
    print_result && println("GPForecasting")
    print_result && println("-------------")
    gp = GPForecasting.GP(m, k)
    before = condition(gp, Observed(x_train), y_train)
    μ = before.m(Latent(x_test))
    σ²= diag(before.k(Latent(x_test)))
    Σ = before.k(Observed(x_test))
    
    print_result && println("Before => MLL_cov(train): ", -logpdf(gp, Observed(x_train), y_train))
    print_result && println("Before => MSE(test): ", mse(μ, y_test))
    print_result && println("Before => MLL_var(test, f): ", mll_var(μ, σ², y_test))
    print_result && println("Before => MLL_cov(test, y): ", mll_cov(μ, Σ, y_test))
    if opt
        pgp = learn(gp, Observed(x_train), y_train, objective, trace=false)
        after = condition(pgp, Observed(x_train), y_train)
        μ = after.m(Latent(x_test))
        σ²= diag(after.k(Latent(x_test)))
        Σ = after.k(Observed(x_test))
        print_result && println("-------------")
        print_result && println("After => MLL_cov(train): ", -logpdf(pgp, Observed(x_train), y_train))
        print_result && println("After => MSE(test): ", mse(μ, y_test))
        print_result && println("After => MLL_var(test, f): ", mll_var(μ, σ², y_test))
        print_result && println("Aftyer => MLL_cov(test, y): ", mll_cov(μ, Σ, y_test))
    end
end

function gaussianprocesses(m, k, logNoise, x_train, y_train, x_test, y_test; print_result=true, opt=true)
    print_result && println("GaussianProcesses")
    print_result && println("-----------------")
    gp = GaussianProcesses.GP(x_train, y_train, m, k, logNoise)
    before = deepcopy(gp)
    μ, σ²= predict_f(before, x_test)
    _, Σ = predict_y(before, x_test, full_cov=true)
    print_result && println("Before => MLL_cov(train): ", -before.mll)
    print_result && println("Before => MSE(test): ", mse(μ, y_test))
    print_result && println("Before => MLL_var(test, f): ", mll_var(μ, σ², y_test))
    print_result && println("Before => MLL_cov(test, y): ", mll_cov(μ, Σ, y_test))
    if opt
        after = deepcopy(gp)
        learn!(after, trace=false)
        #optimize!(after) # using GaussianProcesses' own optimizer
        μ, σ²= predict_f(after, x_test)
        _, Σ = predict_y(after, x_test, full_cov=true)
        print_result && println("-----------------")
        print_result && println("After => MLL_cov(train): ", -after.mll)
        print_result && println("After => MSE(test): ", mse(μ, y_test))
        print_result && println("After => MLL_var(test, f) after: ", mll_var(μ, σ², y_test))
        print_result && println("After => MLL_cov(test, y) after: ", mll_cov(μ, Σ, y_test))
    end
end

function gpy(k, noise_var, x_train, y_train, x_test, y_test; print_result=true, opt=true)
    print_result && println("GPy")
    print_result && println("---")
    gp = GPy.models.GPRegression(x_train, y_train, k, noise_var=noise_var)
    μ, σ²= gp.predict_noiseless(x_test)
    _, Σ = gp.predict(x_test, full_cov=true)
    print_result && println("Before => MLL_cov(train): ", gp.objective_function())
    print_result && println("Before => MSE(test): ", mse(μ, y_test))
    print_result && println("Before => MLL_var(test, f): ", mll_var(μ, σ², y_test))
    print_result && println("Before => MLL_cov(test, y): ", mll_cov(μ, Σ, y_test))
    if opt
        gp.optimize() # using GPy's own optimizer
        #learn!(gp, trace=false)
        μ, σ²= gp.predict_noiseless(x_test)
        _, Σ = gp.predict(x_test, full_cov=true)
        print_result && println("---")
        print_result && println("After => MLL_cov(train): ", gp.objective_function())
        print_result && println("After => MSE(test): ", mse(μ, y_test))
        print_result && println("After => MLL_var(test, f): ", mll_var(μ, σ², y_test))
        print_result && println("After => MLL_cov(test, y): ", mll_cov(μ, Σ, y_test))
    end
end
;

## 1. One dimensional problems

In [24]:
seed!(314159265)                                                                     
n = 50                                                                               
f_1d(x) = sin.(x) .* sin.(2.0 * x) + 0.25 * rand(n)                                  
x_train = sort(2pi .* rand(n))                                                       
x_test = sort(2pi .* rand(n))                                                        
y_train = f_1d(x_train)                                                              
y_test = f_1d(x_test)
;

### 1.1. Squared exponential kernel

In [9]:
k = NoiseKernel(2.0 * (EQ() ▷ 10.0), 0.001 * DiagonalKernel())
m = ZeroMean()
gpforecasting(m, k, x_train, y_train, x_test, y_test)
@benchmark gpforecasting(m, k, x_train, y_train, x_test, y_test, print_result=false, opt=false)

GPForecasting
-------------
Before => MLL_cov(train): 3766.980240219634
Before => MSE(test): 0.14825538985254585
Before => MLL_var(test, f): 1476.7191748817445
Before => MLL_cov(test, y): 3407.3765660334084
-------------
After => MLL_cov(train): -31.35943168820105
After => MSE(test): 0.006339639909682949
After => MLL_var(test, f): -0.5729040630044968
Aftyer => MLL_cov(test, y): -57.941165255286776


BenchmarkTools.Trial: 
  memory estimate:  1.69 MiB
  allocs estimate:  15814
  --------------
  minimum time:     814.850 μs (0.00% GC)
  median time:      893.378 μs (0.00% GC)
  mean time:        1.176 ms (23.24% GC)
  maximum time:     5.489 ms (82.45% GC)
  --------------
  samples:          4205
  evals/sample:     1

In [10]:
k = SEIso(log(10.0), log(sqrt(2.0)))
m = MeanZero()
logNoise = log(sqrt(0.001))
gaussianprocesses(m, k, logNoise, x_train, y_train, x_test, y_test)
@benchmark gaussianprocesses(m, k, logNoise, x_train, y_train, x_test, y_test, print_result=false, opt=false)

GaussianProcesses
-----------------
Before => MLL_cov(train): 3770.787189482617
Before => MSE(test): 0.1482528094589627
Before => MLL_var(test, f): 1478.0822369108082
Before => MLL_cov(test, y): 3407.407348344549
-----------------
After => MLL_cov(train): -31.359431686681283
After => MSE(test): 0.006339643004401141
After => MLL_var(test, f) after: -0.5729201944785125
After => MLL_cov(test, y) after: -57.940584472605366


BenchmarkTools.Trial: 
  memory estimate:  359.33 KiB
  allocs estimate:  1091
  --------------
  minimum time:     546.425 μs (0.00% GC)
  median time:      600.574 μs (0.00% GC)
  mean time:        668.428 μs (8.31% GC)
  maximum time:     4.788 ms (83.71% GC)
  --------------
  samples:          7435
  evals/sample:     1

In [11]:
k = GPy.kern.RBF(input_dim=1, variance=2.0, lengthscale=10.0)
noise_var = 0.001
gpy(k, noise_var, reshape(x_train, (length(x_train), 1)), reshape(y_train, (length(y_train), 1)),
reshape(x_test, (length(x_test), 1)), reshape(y_test, (length(y_test), 1)))
@benchmark gpy(k, noise_var, reshape(x_train, (length(x_train), 1)), reshape(y_train, (length(y_train), 1)),
reshape(x_test, (length(x_test), 1)), reshape(y_test, (length(y_test), 1)), print_result=false, opt=false)

GPy
---
Before => MLL_cov(train): 3770.7490822655695
Before => MSE(test): 0.1482528352712732
Before => MLL_var(test, f): 1478.0685933599136
Before => MLL_cov(test, y): 3407.4072433172787
---
After => MLL_cov(train): -31.359431688267463
After => MSE(test): 0.0063396402095871095
After => MLL_var(test, f): -0.5729058969244587
After => MLL_cov(test, y): -57.94063600622842


BenchmarkTools.Trial: 
  memory estimate:  613.75 KiB
  allocs estimate:  21483
  --------------
  minimum time:     11.965 ms (0.00% GC)
  median time:      12.199 ms (0.00% GC)
  mean time:        12.629 ms (1.17% GC)
  maximum time:     32.667 ms (0.00% GC)
  --------------
  samples:          396
  evals/sample:     1

### 1.2. Standard periodic kernel

In [12]:
k = NoiseKernel(2.0 * periodicise(EQ() ▷ 10.0, 2π), 0.001 * DiagonalKernel())
m = ZeroMean()
gpforecasting(m, k, x_train, y_train, x_test, y_test)
@benchmark gpforecasting(m, k, x_train, y_train, x_test, y_test, print_result=false, opt=false)

GPForecasting
-------------
Before => MLL_cov(train): 2778.2568773410835
Before => MSE(test): 0.13079765742455027
Before => MLL_var(test, f): 760.2721834289244
Before => MLL_cov(test, y): 3021.85514135819
-------------
After => MLL_cov(train): -28.808586064107345
After => MSE(test): 0.010108277078175808
After => MLL_var(test, f): 0.3649906189726084
Aftyer => MLL_cov(test, y): -45.12023648359445


BenchmarkTools.Trial: 
  memory estimate:  1.73 MiB
  allocs estimate:  16144
  --------------
  minimum time:     934.623 μs (0.00% GC)
  median time:      1.037 ms (0.00% GC)
  mean time:        1.425 ms (24.52% GC)
  maximum time:     7.199 ms (70.31% GC)
  --------------
  samples:          3497
  evals/sample:     1

In [13]:
k = Periodic(log(10.0), log(sqrt(2.0)), log(2π))
m = MeanZero()
logNoise = log(sqrt(0.001))
gaussianprocesses(m, k, logNoise, x_train, y_train, x_test, y_test)
@benchmark gaussianprocesses(m, k, logNoise, x_train, y_train, x_test, y_test, print_result=false, opt=false)

GaussianProcesses
-----------------
Before => MLL_cov(train): 2781.067999039825
Before => MSE(test): 0.13080137978135378
Before => MLL_var(test, f): 760.9553814833481
Before => MLL_cov(test, y): 3021.956394744089
-----------------
After => MLL_cov(train): -28.808587160781165
After => MSE(test): 0.01010866495058404
After => MLL_var(test, f) after: 0.3648342632338637
After => MLL_cov(test, y) after: -45.11967967384456


BenchmarkTools.Trial: 
  memory estimate:  359.36 KiB
  allocs estimate:  1092
  --------------
  minimum time:     742.127 μs (0.00% GC)
  median time:      797.971 μs (0.00% GC)
  mean time:        876.749 μs (6.90% GC)
  maximum time:     5.199 ms (77.77% GC)
  --------------
  samples:          5672
  evals/sample:     1

In [25]:
k = GPy.kern.StdPeriodic(input_dim=1, variance=2.0, lengthscale=10.0, period=2π)
noise_var = 0.001
gpy(k, noise_var, reshape(x_train, (length(x_train), 1)), reshape(y_train, (length(y_train), 1)),
reshape(x_test, (length(x_test), 1)), reshape(y_test, (length(y_test), 1)))
@benchmark gpy(k, noise_var, reshape(x_train, (length(x_train), 1)), reshape(y_train, (length(y_train), 1)),
reshape(x_test, (length(x_test), 1)), reshape(y_test, (length(y_test), 1)), print_result=false, opt=false)

GPy
---
Before => MLL_cov(train): 2890.2312291334088
Before => MSE(test): 0.12531242054651615
Before => MLL_var(test, f): 973.2272382866464
Before => MLL_cov(test, y): 2981.772001725934
---
After => MLL_cov(train): 6.8792682396340314
After => MSE(test): 0.028511919017709032
After => MLL_var(test, f): 0.3445370710768054
After => MLL_cov(test, y): -8.529002686282148


BenchmarkTools.Trial: 
  memory estimate:  613.75 KiB
  allocs estimate:  21483
  --------------
  minimum time:     11.933 ms (0.00% GC)
  median time:      12.191 ms (0.00% GC)
  mean time:        12.499 ms (1.48% GC)
  maximum time:     20.052 ms (35.43% GC)
  --------------
  samples:          401
  evals/sample:     1

### 1.3. Rational quadratic kernel

In [15]:
k = NoiseKernel(2.0 * GPForecasting.RQ(2.0) ▷ 5.0, 0.001 * DiagonalKernel())
m = ZeroMean()
gpforecasting(m, k, x_train, y_train, x_test, y_test)
@benchmark gpforecasting(m, k, x_train, y_train, x_test, y_test, print_result=false, opt=false)

GPForecasting
-------------
Before => MLL_cov(train): 2491.876499557334
Before => MSE(test): 0.12226859947646133
Before => MLL_var(test, f): 771.9774842426579
Before => MLL_cov(test, y): 2880.4369702043728
-------------
After => MLL_cov(train): -31.359431327439474
After => MSE(test): 0.006339625546752299
After => MLL_var(test, f): -0.5728913634245152
Aftyer => MLL_cov(test, y): -57.94114980835164


BenchmarkTools.Trial: 
  memory estimate:  1.69 MiB
  allocs estimate:  15868
  --------------
  minimum time:     1.088 ms (0.00% GC)
  median time:      1.255 ms (0.00% GC)
  mean time:        1.677 ms (21.39% GC)
  maximum time:     7.869 ms (75.64% GC)
  --------------
  samples:          2970
  evals/sample:     1

In [16]:
k = GaussianProcesses.RQIso(log(5.0), log(sqrt(2.0)), log(2.0))
m = MeanZero()
logNoise = log(sqrt(0.001))
gaussianprocesses(m, k, logNoise, x_train, y_train, x_test, y_test)
@benchmark gaussianprocesses(m, k, logNoise, x_train, y_train, x_test, y_test, print_result=false, opt=false)

GaussianProcesses
-----------------
Before => MLL_cov(train): 2494.2092851510424
Before => MSE(test): 0.12227096361627385
Before => MLL_var(test, f): 772.7003788248147
Before => MLL_cov(test, y): 2880.4871925198668
-----------------
After => MLL_cov(train): -31.359431228470633
After => MSE(test): 0.006339596733726229
After => MLL_var(test, f) after: -0.5729204923449185
After => MLL_cov(test, y) after: -57.940598476025116


BenchmarkTools.Trial: 
  memory estimate:  359.36 KiB
  allocs estimate:  1092
  --------------
  minimum time:     761.845 μs (0.00% GC)
  median time:      852.425 μs (0.00% GC)
  mean time:        954.531 μs (6.77% GC)
  maximum time:     5.638 ms (78.96% GC)
  --------------
  samples:          5207
  evals/sample:     1

In [17]:
k = GPy.kern.RatQuad(input_dim=1, variance=2.0, lengthscale=5.0, power=2.0)
noise_var = 0.001
gpy(k, noise_var, reshape(x_train, (length(x_train), 1)), reshape(y_train, (length(y_train), 1)),
reshape(x_test, (length(x_test), 1)), reshape(y_test, (length(y_test), 1)))
@benchmark gpy(k, noise_var, reshape(x_train, (length(x_train), 1)), reshape(y_train, (length(y_train), 1)),
reshape(x_test, (length(x_test), 1)), reshape(y_test, (length(y_test), 1)), print_result=false, opt=false)

GPy
---
Before => MLL_cov(train): 2039.8173847552514
Before => MSE(test): 0.11002642210716784
Before => MLL_var(test, f): 535.4312750107787
Before => MLL_cov(test, y): 2231.446306080221
---
After => MLL_cov(train): -31.35941015524378
After => MSE(test): 0.006339556891899399
After => MLL_var(test, f): -0.5728882631466432
After => MLL_cov(test, y): -57.94062702287771


BenchmarkTools.Trial: 
  memory estimate:  613.75 KiB
  allocs estimate:  21483
  --------------
  minimum time:     12.591 ms (0.00% GC)
  median time:      12.848 ms (0.00% GC)
  mean time:        13.149 ms (1.25% GC)
  maximum time:     20.387 ms (30.94% GC)
  --------------
  samples:          381
  evals/sample:     1

### 1.4. Mattern 5/2 kernel

In [18]:
k = NoiseKernel(2.0 * MA(5/2) ▷ 10.0, 0.001 * DiagonalKernel())
m = ConstantMean(0.0)
gpforecasting(m, k, x_train, y_train, x_test, y_test)
@benchmark gpforecasting(m, k, x_train, y_train, x_test, y_test, print_result=false, opt=false)

GPForecasting
-------------
Before => MLL_cov(train): 2556.159032671505
Before => MSE(test): 0.11075488248840884
Before => MLL_var(test, f): 625.1263034476407
Before => MLL_cov(test, y): 2337.0599977777874
-------------
After => MLL_cov(train): -28.755471135797887
After => MSE(test): 0.005838031744599887
After => MLL_var(test, f): -0.8934805590282198
Aftyer => MLL_cov(test, y): -56.414629998655


BenchmarkTools.Trial: 
  memory estimate:  2.15 MiB
  allocs estimate:  15880
  --------------
  minimum time:     925.325 μs (0.00% GC)
  median time:      1.054 ms (0.00% GC)
  mean time:        1.526 ms (30.76% GC)
  maximum time:     7.082 ms (79.14% GC)
  --------------
  samples:          3267
  evals/sample:     1

In [19]:
k = Mat52Iso(log(10.0), log(sqrt(2.0)))
m = MeanZero()
logNoise = log(sqrt(0.001))
gaussianprocesses(m, k, logNoise, x_train, y_train, x_test, y_test)
@benchmark gaussianprocesses(m, k, logNoise, x_train, y_train, x_test, y_test, print_result=false, opt=false)

GaussianProcesses
-----------------
Before => MLL_cov(train): 2558.3621232306436
Before => MSE(test): 0.11074141340154292
Before => MLL_var(test, f): 625.5639745229428
Before => MLL_cov(test, y): 2336.7710024989365
-----------------
After => MLL_cov(train): -28.755471135633094
After => MSE(test): 0.005838029243976166
After => MLL_var(test, f) after: -0.8934811076285538
After => MLL_cov(test, y) after: -56.41427396434535


BenchmarkTools.Trial: 
  memory estimate:  359.23 KiB
  allocs estimate:  1087
  --------------
  minimum time:     431.911 μs (0.00% GC)
  median time:      475.841 μs (0.00% GC)
  mean time:        550.778 μs (12.28% GC)
  maximum time:     5.621 ms (91.35% GC)
  --------------
  samples:          9006
  evals/sample:     1

In [20]:
k = GPy.kern.Matern52(input_dim=1, variance=2.0, lengthscale=10.0)
noise_var = 0.001
gpy(k, noise_var, reshape(x_train, (length(x_train), 1)), reshape(y_train, (length(y_train), 1)),
reshape(x_test, (length(x_test), 1)), reshape(y_test, (length(y_test), 1)))
@benchmark gpy(k, noise_var, reshape(x_train, (length(x_train), 1)), reshape(y_train, (length(y_train), 1)),
reshape(x_test, (length(x_test), 1)), reshape(y_test, (length(y_test), 1)), print_result=false, opt=false)

GPy
---
Before => MLL_cov(train): 2558.3400729079895
Before => MSE(test): 0.11074154820919034
Before => MLL_var(test, f): 625.5595945179847
Before => MLL_cov(test, y): 2336.7738949917907
---
After => MLL_cov(train): -28.7554711357821
After => MSE(test): 0.005838031403802246
After => MLL_var(test, f): -0.893480839048405
After => MLL_cov(test, y): -56.41427771607259


BenchmarkTools.Trial: 
  memory estimate:  613.75 KiB
  allocs estimate:  21483
  --------------
  minimum time:     12.076 ms (0.00% GC)
  median time:      12.460 ms (0.00% GC)
  mean time:        13.173 ms (1.33% GC)
  maximum time:     22.411 ms (29.93% GC)
  --------------
  samples:          380
  evals/sample:     1

### 1.5. Mixed kernel

In [21]:
k1 = EQ() ▷ 10.0
k2 = periodicise(EQ() ▷ 10.0, 2π)
k3 = GPForecasting.RQ(2.0) ▷ 10.0
k4 = MA(5/2) ▷ 10.0
k = NoiseKernel(2.0 * (k1 + k2 * k3) + k4, 0.001 * DiagonalKernel())
m = ZeroMean()
gpforecasting(m, k, x_train, y_train, x_test, y_test)
@benchmark gpforecasting(m, k, x_train, y_train, x_test, y_test, print_result=false, opt=false)

GPForecasting
-------------
Before => MLL_cov(train): 2011.7433463645914
Before => MSE(test): 0.10434757275194265
Before => MLL_var(test, f): 514.815138737102
Before => MLL_cov(test, y): 2065.663581831527
-------------
After => MLL_cov(train): -36.93919046063505
After => MSE(test): 0.005521044170657068
After => MLL_var(test, f): -0.21247785163171296
Aftyer => MLL_cov(test, y): -60.413218850085556


BenchmarkTools.Trial: 
  memory estimate:  3.64 MiB
  allocs estimate:  16738
  --------------
  minimum time:     1.827 ms (0.00% GC)
  median time:      2.018 ms (0.00% GC)
  mean time:        2.840 ms (28.20% GC)
  maximum time:     8.763 ms (63.80% GC)
  --------------
  samples:          1758
  evals/sample:     1

In [22]:
k1 = fix(SEIso(log(10.0), log(sqrt(1.0))), :lσ)
k2 = fix(Periodic(log(10.0), log(sqrt(1.0)), log(2π)), :lσ)
k3 = fix(GaussianProcesses.RQIso(log(10.0), log(sqrt(1.0)), log(2.0)), :lσ)
k4 = fix(Mat52Iso(log(10.0), log(sqrt(1.0))), :lσ)
k = Const(log(sqrt(2.0))) * (k1 + k2 * k3) + k4
m = MeanZero()
logNoise = log(sqrt(0.001))
gaussianprocesses(m, k, logNoise, x_train, y_train, x_test, y_test)
@benchmark gaussianprocesses(m, k, logNoise, x_train, y_train, x_test, y_test, print_result=false, opt=false)

GaussianProcesses
-----------------
Before => MLL_cov(train): 2013.5208296831067
Before => MSE(test): 0.10432954414880101
Before => MLL_var(test, f): 515.1802436470141
Before => MLL_cov(test, y): 2065.35969305844
-----------------
After => MLL_cov(train): -32.53201895418004
After => MSE(test): 0.006164229768577374
After => MLL_var(test, f) after: -0.4528893378885508
After => MLL_cov(test, y) after: -59.18062065053402


BenchmarkTools.Trial: 
  memory estimate:  2.11 MiB
  allocs estimate:  36158
  --------------
  minimum time:     6.073 ms (0.00% GC)
  median time:      6.243 ms (0.00% GC)
  mean time:        6.865 ms (8.30% GC)
  maximum time:     15.673 ms (47.40% GC)
  --------------
  samples:          728
  evals/sample:     1

In [26]:
k1 = GPy.kern.RBF(input_dim=1, variance=1.0, lengthscale=10.0)
k1."variance".fix()
k2 = GPy.kern.StdPeriodic(input_dim=1, variance=1.0, lengthscale=10.0, period=2π)
k2."variance".fix()
k3 = GPy.kern.RatQuad(input_dim=1, variance=1.0, lengthscale=10.0, power=2.0)
k3."variance".fix()
k4 = GPy.kern.Matern52(input_dim=1, variance=1.0, lengthscale=10.0)
k4."variance".fix()
k = GPy.kern.Bias(input_dim=1, variance=2.0) * (k1 + k2 * k3) + k4
noise_var = 0.001
gpy(k, noise_var, reshape(x_train, (length(x_train), 1)), reshape(y_train, (length(y_train), 1)),
reshape(x_test, (length(x_test), 1)), reshape(y_test, (length(y_test), 1)))
@benchmark gpy(k, noise_var, reshape(x_train, (length(x_train), 1)), reshape(y_train, (length(y_train), 1)),
reshape(x_test, (length(x_test), 1)), reshape(y_test, (length(y_test), 1)), print_result=false, opt=false)

GPy
---
Before => MLL_cov(train): 2249.8190982185083
Before => MSE(test): 0.11628337862057529
Before => MLL_var(test, f): 632.2348583879655
Before => MLL_cov(test, y): 2505.5798844922924
---
After => MLL_cov(train): -32.53202100958151
After => MSE(test): 0.006163553199038626
After => MLL_var(test, f): -0.4528906194516189
After => MLL_cov(test, y): -59.18111658414189


BenchmarkTools.Trial: 
  memory estimate:  613.75 KiB
  allocs estimate:  21483
  --------------
  minimum time:     25.599 ms (0.00% GC)
  median time:      26.121 ms (0.00% GC)
  mean time:        27.177 ms (0.70% GC)
  maximum time:     38.327 ms (19.83% GC)
  --------------
  samples:          185
  evals/sample:     1

## 2. Multidimensional input problems

In [27]:
seed!(314159265)                                                                     
n = 100                                                                              
d = 3                                                                                
s = collect(1.0:1.0:d)                                                               
f_Md(x) = prod(sin.(x .* s'), dims=2)[:] + 0.1 * rand(n)                             
x_train = 2pi .* rand(n, d)      
x_test = 2pi .* rand(n, d)
y_train = f_Md(x_train)
y_test = f_Md(x_test)
;

### 2.1. Squared exponential kernel

In [28]:
k = NoiseKernel(1.0 * (EQ() ▷ [2.0 for i=1:d]), 0.001 * DiagonalKernel())
m = ZeroMean()
gpforecasting(m, k, x_train, y_train, x_test, y_test)
@benchmark gpforecasting(m, k, x_train, y_train, x_test, y_test, print_result=false, opt=false)

GPForecasting
-------------
Before => MLL_cov(train): 1276.191287482529
Before => MSE(test): 0.4071110749424119
Before => MLL_var(test, f): 45.884759655685286
Before => MLL_cov(test, y): 4419.245955870601
-------------
After => MLL_cov(train): 27.81058036431854
After => MSE(test): 0.09809995063117481
After => MLL_var(test, f): 0.22530959375093262
Aftyer => MLL_cov(test, y): 18.396578851654084


BenchmarkTools.Trial: 
  memory estimate:  7.12 MiB
  allocs estimate:  61327
  --------------
  minimum time:     5.394 ms (0.00% GC)
  median time:      6.084 ms (0.00% GC)
  mean time:        8.032 ms (25.18% GC)
  maximum time:     14.913 ms (49.85% GC)
  --------------
  samples:          622
  evals/sample:     1

In [29]:
k = SEArd(log.([2.0 for i=1:d]), log(sqrt(1.0)))
m = MeanZero()
logNoise = log(sqrt(0.001))
gaussianprocesses(m, k, logNoise, x_train', y_train, x_test', y_test)
@benchmark gaussianprocesses(m, k, logNoise, x_train', y_train, x_test', y_test, print_result=false, opt=false)

GaussianProcesses
-----------------
Before => MLL_cov(train): 1276.965877443792
Before => MSE(test): 0.4072840523764735
Before => MLL_var(test, f): 45.91742408278396
Before => MLL_cov(test, y): 4420.701235673641
-----------------
After => MLL_cov(train): 27.810561094145072
After => MSE(test): 0.09810226298497927
After => MLL_var(test, f) after: 0.22531958433643018
After => MLL_cov(test, y) after: 18.398221464454195


BenchmarkTools.Trial: 
  memory estimate:  2.16 MiB
  allocs estimate:  4131
  --------------
  minimum time:     2.594 ms (0.00% GC)
  median time:      3.446 ms (0.00% GC)
  mean time:        4.004 ms (13.66% GC)
  maximum time:     10.643 ms (61.74% GC)
  --------------
  samples:          1246
  evals/sample:     1

In [30]:
k = GPy.kern.RBF(input_dim=d, variance=1.0, ARD=true, lengthscale=[2.0 for i=1:d])
noise_var = 0.001
gpy(k, noise_var, x_train, reshape(y_train, (length(y_train), 1)),
x_test, reshape(y_test, (length(y_test), 1)))
@benchmark gpy(k, noise_var, x_train, reshape(y_train, (length(y_train), 1)),
x_test, reshape(y_test, (length(y_test), 1)), print_result=false, opt=false)

GPy
---
Before => MLL_cov(train): 1276.9581261044007
Before => MSE(test): 0.4072823214890315
Before => MLL_var(test, f): 45.91709718755408
Before => MLL_cov(test, y): 4420.686673190201
---
After => MLL_cov(train): 27.8105609721553
After => MSE(test): 0.09810350098016432
After => MLL_var(test, f): 0.22532671776614194
After => MLL_cov(test, y): 18.399268719614085


BenchmarkTools.Trial: 
  memory estimate:  2.29 MiB
  allocs estimate:  82678
  --------------
  minimum time:     31.116 ms (0.00% GC)
  median time:      33.522 ms (0.00% GC)
  mean time:        34.779 ms (2.03% GC)
  maximum time:     48.007 ms (13.91% GC)
  --------------
  samples:          144
  evals/sample:     1

### 2.2. Standard periodic kernel (not implemented in GaussianProcesses )

In [31]:
k = NoiseKernel(2.0 * periodicise(EQ() ▷ [5.0 for i=1:d], [2π for i=1:d]), 0.001 * DiagonalKernel())
m = ZeroMean()
gpforecasting(m, k, x_train, y_train, x_test, y_test)
@benchmark gpforecasting(m, k, x_train, y_train, x_test, y_test, print_result=false, opt=false)

GPForecasting
-------------
Before => MLL_cov(train): 3379.8734617736277
Before => MSE(test): 0.15109565238928888
Before => MLL_var(test, f): 144.4562307160178
Before => MLL_cov(test, y): 5304.246158359595
-------------
After => MLL_cov(train): 34.36332033669985
After => MSE(test): 0.12005882786041555
After => MLL_var(test, f): 29.256882478772546
Aftyer => MLL_cov(test, y): 36.582892470865325


BenchmarkTools.Trial: 
  memory estimate:  7.31 MiB
  allocs estimate:  61567
  --------------
  minimum time:     5.396 ms (0.00% GC)
  median time:      6.375 ms (0.00% GC)
  mean time:        8.292 ms (25.93% GC)
  maximum time:     14.900 ms (45.76% GC)
  --------------
  samples:          603
  evals/sample:     1

In [33]:
k = GPy.kern.StdPeriodic(input_dim=d, variance=2.0, lengthscale=[5.0 for i=1:d], period=[2π for i=1:d], ARD1=true, ARD2=true)
noise_var = 0.001
gpy(k, noise_var, x_train, reshape(y_train, (length(y_train), 1)),
x_test, reshape(y_test, (length(y_test), 1)))
@benchmark gpy(k, noise_var, x_train, reshape(y_train, (length(y_train), 1)),
x_test, reshape(y_test, (length(y_test), 1)), print_result=false, opt=false)

GPy
---
Before => MLL_cov(train): 4146.115608870026
Before => MSE(test): 0.13734145079774088
Before => MLL_var(test, f): 283.10978364091284
Before => MLL_cov(test, y): 5575.602738375718
---
After => MLL_cov(train): 34.38181803151828
After => MSE(test): 0.11992910741051521
After => MLL_var(test, f): 34.85096435475269
After => MLL_cov(test, y): 36.38247496900728


BenchmarkTools.Trial: 
  memory estimate:  2.29 MiB
  allocs estimate:  82678
  --------------
  minimum time:     33.942 ms (0.00% GC)
  median time:      35.422 ms (0.00% GC)
  mean time:        36.691 ms (1.94% GC)
  maximum time:     47.563 ms (15.78% GC)
  --------------
  samples:          137
  evals/sample:     1

### 2.3. Rational quadratic kernel

In [34]:
k = NoiseKernel(2.0 * (GPForecasting.RQ(Fixed(2.0)) ▷ [2.0 for i=1:d]), 0.001 * DiagonalKernel())
m = ZeroMean()
gpforecasting(m, k, x_train, y_train, x_test, y_test)
@benchmark gpforecasting(m, k, x_train, y_train, x_test, y_test, print_result=false, opt=false)

GPForecasting
-------------
Before => MLL_cov(train): 300.0782662907241
Before => MSE(test): 0.45069841363296925
Before => MLL_var(test, f): 8.10416742783544
Before => MLL_cov(test, y): 1174.4605087920281
-------------
After => MLL_cov(train): 28.972434450264238
After => MSE(test): 0.10054178097161147
After => MLL_var(test, f): 0.24119368608041608
Aftyer => MLL_cov(test, y): 20.976406720435826


BenchmarkTools.Trial: 
  memory estimate:  7.12 MiB
  allocs estimate:  61387
  --------------
  minimum time:     6.275 ms (0.00% GC)
  median time:      7.223 ms (0.00% GC)
  mean time:        9.093 ms (22.94% GC)
  maximum time:     15.859 ms (44.96% GC)
  --------------
  samples:          549
  evals/sample:     1

In [35]:
k = fix(GaussianProcesses.RQArd(log.([2.0 for i=1:d]), log(sqrt(2.0)), log(2.0)), :lα)
m = MeanZero()
logNoise = log(sqrt(0.001))
gaussianprocesses(m, k, logNoise, x_train', y_train, x_test', y_test)
@benchmark gaussianprocesses(m, k, logNoise, x_train', y_train, x_test', y_test, print_result=false, opt=false)

GaussianProcesses
-----------------
Before => MLL_cov(train): 300.1572454511439
Before => MSE(test): 0.4507881156857738
Before => MLL_var(test, f): 8.106859242015442
Before => MLL_cov(test, y): 1174.7332322552386
-----------------
After => MLL_cov(train): 28.972380333955606
After => MSE(test): 0.10054441929487325
After => MLL_var(test, f) after: 0.24121140115477602
After => MLL_cov(test, y) after: 20.978591875637335


BenchmarkTools.Trial: 
  memory estimate:  2.16 MiB
  allocs estimate:  4130
  --------------
  minimum time:     3.086 ms (0.00% GC)
  median time:      4.049 ms (0.00% GC)
  mean time:        4.690 ms (12.31% GC)
  maximum time:     11.884 ms (54.96% GC)
  --------------
  samples:          1064
  evals/sample:     1

In [36]:
k = GPy.kern.RatQuad(input_dim=d, variance=2.0, lengthscale=[2.0 for i=1:d], power=2.0, ARD=true)
k."power".fix()
noise_var = 0.001
gpy(k, noise_var, x_train, reshape(y_train, (length(y_train), 1)),
x_test, reshape(y_test, (length(y_test), 1)))
@benchmark gpy(k, noise_var, x_train, reshape(y_train, (length(y_train), 1)),
x_test, reshape(y_test, (length(y_test), 1)), print_result=false, opt=false)

GPy
---
Before => MLL_cov(train): 105.47566718275176
Before => MSE(test): 0.3033706362566648
Before => MLL_var(test, f): 1.013819924867406
Before => MLL_cov(test, y): 118.02481512045507
---
After => MLL_cov(train): 28.972380046023126
After => MSE(test): 0.1005416232586522
After => MLL_var(test, f): 0.2411928166652953
After => MLL_cov(test, y): 20.976250753927943


BenchmarkTools.Trial: 
  memory estimate:  2.29 MiB
  allocs estimate:  82678
  --------------
  minimum time:     32.396 ms (0.00% GC)
  median time:      34.351 ms (0.00% GC)
  mean time:        35.518 ms (1.96% GC)
  maximum time:     45.975 ms (15.79% GC)
  --------------
  samples:          141
  evals/sample:     1

### 2.4. Mattern 5/2 kernel

In [37]:
k = NoiseKernel(2.0 * (MA(5/2) ▷ [2.0 for i=1:d]), 0.001 * DiagonalKernel())
m = ZeroMean()
gpforecasting(m, k, x_train, y_train, x_test, y_test)
@benchmark gpforecasting(m, k, x_train, y_train, x_test, y_test, print_result=false, opt=false)

GPForecasting
-------------
Before => MLL_cov(train): 104.94568163481921
Before => MSE(test): 0.2387315118745936
Before => MLL_var(test, f): 0.875735261494879
Before => MLL_cov(test, y): 93.0163136262823
-------------
After => MLL_cov(train): 28.50115756187644
After => MSE(test): 0.09881765784592228
After => MLL_var(test, f): 0.23168536130867723
Aftyer => MLL_cov(test, y): 19.69022557212292


BenchmarkTools.Trial: 
  memory estimate:  8.95 MiB
  allocs estimate:  61399
  --------------
  minimum time:     6.783 ms (0.00% GC)
  median time:      8.386 ms (0.00% GC)
  mean time:        10.281 ms (26.73% GC)
  maximum time:     17.658 ms (44.55% GC)
  --------------
  samples:          486
  evals/sample:     1

In [38]:
k = Mat52Ard(log.([2.0 for i=1:d]), log(sqrt(2.0)))
m = MeanZero()
logNoise = log(sqrt(0.001))
gaussianprocesses(m, k, logNoise, x_train', y_train, x_test', y_test)
@benchmark gaussianprocesses(m, k, logNoise, x_train', y_train, x_test', y_test, print_result=false, opt=false)

GaussianProcesses
-----------------
Before => MLL_cov(train): 104.95021522614886
Before => MSE(test): 0.2387415002455038
Before => MLL_var(test, f): 0.8757821859749355
Before => MLL_cov(test, y): 93.01808773030875
-----------------
After => MLL_cov(train): 28.501096338931987
After => MSE(test): 0.09881866390272447
After => MLL_var(test, f) after: 0.23169090100484718
After => MLL_cov(test, y) after: 19.69104551362154


BenchmarkTools.Trial: 
  memory estimate:  2.16 MiB
  allocs estimate:  4131
  --------------
  minimum time:     2.871 ms (0.00% GC)
  median time:      3.712 ms (0.00% GC)
  mean time:        4.280 ms (13.27% GC)
  maximum time:     10.860 ms (56.98% GC)
  --------------
  samples:          1166
  evals/sample:     1

In [39]:
k = GPy.kern.Matern52(input_dim=d, variance=2.0, lengthscale=[2.0 for i=1:d], ARD=true)
noise_var = 0.001
gpy(k, noise_var, x_train, reshape(y_train, (length(y_train), 1)),
x_test, reshape(y_test, (length(y_test), 1)))
@benchmark gpy(k, noise_var, x_train, reshape(y_train, (length(y_train), 1)),
x_test, reshape(y_test, (length(y_test), 1)), print_result=false, opt=false)

GPy
---
Before => MLL_cov(train): 104.95016988130722
Before => MSE(test): 0.2387414003320791
Before => MLL_var(test, f): 0.8757817166032631
Before => MLL_cov(test, y): 93.01806998730181
---
After => MLL_cov(train): 28.501095370926713
After => MSE(test): 0.09881771396260247
After => MLL_var(test, f): 0.23168575878030995
After => MLL_cov(test, y): 19.6902352414579


BenchmarkTools.Trial: 
  memory estimate:  2.29 MiB
  allocs estimate:  82678
  --------------
  minimum time:     31.306 ms (0.00% GC)
  median time:      32.442 ms (0.00% GC)
  mean time:        33.556 ms (2.11% GC)
  maximum time:     43.727 ms (20.07% GC)
  --------------
  samples:          149
  evals/sample:     1

## 3. Multidimensional output problems

In [40]:
seed!(314159265)                                                                             
                                                                                             
n = 100                                                                                      
p = 5                                                                                        
                                                                                             
C = rand(p, p)                                                                               
                                                                                             
x = sort(rand(2n))                                                                           
y = zeros(2n, p)                                                                             
y[1,:] = rand(p)                                                                             
for i=2:2n                                                                                   
    y[i,:] = y[i-1,:] .+ rand(p) .- 0.5                                                      
end                                                                                          
                                                                                             
y = y*C                                                                                      
x_train = x[1:2:end]                                                                         
x_test = x[2:2:end]                                                                          
y_train = y[1:2:end,:]                                                                       
y_test = y[2:2:end,:]                                                                        
mean_y_train = mean(y_train, dims=1)                                                         
std_y_train = std(y_train, dims=1)                                                           
y_train = (y_train .- mean_y_train) ./ std_y_train                                           
y_test = (y_test .- mean_y_train) ./ std_y_train
;

### 3.1.  Identity coregionalization matrix (full rank)

In [41]:
println("GPForecasting")
println("-------------")
H = Eye(p)
gp = GPForecasting.GP(OLMMKernel(p, p, 0.0, [0.001 for i=1:p], H, [(EQ() ▷ 1.0) for i=1:p]))
before = condition(gp, x_train, y_train)
b = before.m(x_test)
println("Before => MLL_cov(train): ", -logpdf(gp, x_train, y_train))
for i=1:p
    println("Before => MSE_$(i)(test): ", mse(b[:,i], y_test[:,i]))
end

gp = learn(gp, x_train, y_train, objective, trace=false)
println("-------------")

after = condition(gp, x_train, y_train)
a = after.m(x_test)
println("After => MLL_cov(train): ", -logpdf(gp, x_train, y_train))
for i=1:p
    println("After => MSE_$(i)(test): ", mse(a[:,i], y_test[:,i]))
end

GPForecasting
-------------
Before => MLL_cov(train): 123269.87302493502
Before => MSE_1(test): 0.44000806383244245
Before => MSE_2(test): 0.5162941864690386
Before => MSE_3(test): 0.4376368048964098
Before => MSE_4(test): 0.4569272147581291
Before => MSE_5(test): 0.4554561556686091
-------------
After => MLL_cov(train): 143.41651616737295
After => MSE_1(test): 0.04083420618349919
After => MSE_2(test): 0.057474087179421
After => MSE_3(test): 0.04195717203591064
After => MSE_4(test): 0.06368191349412453
After => MSE_5(test): 0.0455594276166225


In [43]:
println("GPy")
println("---")
H = Eye(p)
kappa = zeros(5)
K = GPy.kern.RBF(input_dim=1, variance=1.0, lengthscale=1.0)
K."variance".fix()
k = GPy.util.multioutput.ICM(input_dim=1, num_outputs=5, kernel=K, W_rank=5, W=Matrix(H), kappa=kappa)
k."B"."W".fix()
k."B"."kappa".fix()
gp = GPy.models.GPCoregionalizedRegression([reshape(x_train, (n, 1)) for i=1:p], [y_train[:,i:i] for i=1:p], kernel=k)
gp."mixed_noise"."Gaussian_noise_0".variance = 0.001
gp."mixed_noise"."Gaussian_noise_1".variance = 0.001
gp."mixed_noise"."Gaussian_noise_2".variance = 0.001
gp."mixed_noise"."Gaussian_noise_3".variance = 0.001
gp."mixed_noise"."Gaussian_noise_4".variance = 0.001

println("Before => MLL_cov(train): ", gp.objective_function())
for i=1:p
    println("Before => MSE_$(i)(test): ", mse(gp.predict(hcat(x_test, (i-1)*ones(Int64, n)), Y_metadata=Dict("output_index" => [i-1 for j=1:n]))[1], y_test[:,i]))
end

gp.optimize()
println("---")

println("After => MLL_cov(train): ", gp.objective_function())
for i=1:p
    println("After => MSE_$(i)(test): ", mse(gp.predict(hcat(x_test, (i-1)*ones(Int64, n)), Y_metadata=Dict("output_index" => [i-1 for j=1:n]))[1], y_test[:,i]))
end

GPy
---
Before => MLL_cov(train): 123387.29117724793
Before => MSE_1(test): 0.44000808028240757
Before => MSE_2(test): 0.5162942530173277
Before => MSE_3(test): 0.43763686052604156
Before => MSE_4(test): 0.45692737714738757
Before => MSE_5(test): 0.45545617765610297
---
After => MLL_cov(train): 144.9376530219561
After => MSE_1(test): 0.04013653573389523
After => MSE_2(test): 0.06651769443900414
After => MSE_3(test): 0.04148085594498114
After => MSE_4(test): 0.06417540466774445
After => MSE_5(test): 0.04629993565336812


### 3.2. Fixed full rank coregionalization matrix

In [44]:
println("GPForecasting")
println("-------------")
U, S, V = svd(cov(y_train))
H = U * Diagonal(sqrt.(S))
gp = GPForecasting.GP(OLMMKernel(p, p, 0.0, [0.001 for i=1:p], H, [(EQ() ▷ 1.0) for i=1:p]))
before = condition(gp, x_train, y_train)
b = before.m(x_test)
println("Before => MLL_cov(train): ", -logpdf(gp, x_train, y_train))
for i=1:p
    println("Before => MSE_$(i)(test): ", mse(b[:,i], y_test[:,i]))
end

gp = learn(gp, x_train, y_train, objective, trace=false)
println("-------------")

after = condition(gp, x_train, y_train)
a = after.m(x_test)
println("After => MLL_cov(train): ", -logpdf(gp, x_train, y_train))
for i=1:p
    println("After => MSE_$(i)(test): ", mse(a[:,i], y_test[:,i]))
end

GPForecasting
-------------
Before => MLL_cov(train): 120943.86231187363
Before => MSE_1(test): 0.4400080638324424
Before => MSE_2(test): 0.5162941864690385
Before => MSE_3(test): 0.4376368048964098
Before => MSE_4(test): 0.45692721475812936
Before => MSE_5(test): 0.4554561556686091
-------------
After => MLL_cov(train): -257.56556243845984
After => MSE_1(test): 0.04238016066550304
After => MSE_2(test): 0.0596619194006132
After => MSE_3(test): 0.044695518933201485
After => MSE_4(test): 0.06734888377183729
After => MSE_5(test): 0.04607940775581372


In [45]:
println("GPy")
println("---")
U, S, V = svd(cov(y_train))
H = U * Diagonal(sqrt.(S))
kappa = zeros(5)
K = GPy.kern.RBF(input_dim=1, variance=1.0, lengthscale=1.0)
K."variance".fix()
k = GPy.util.multioutput.ICM(input_dim=1, num_outputs=5, kernel=K, W_rank=5, W=H, kappa=kappa)
k."B"."W".fix()
k."B"."kappa".fix()
gp = GPy.models.GPCoregionalizedRegression([reshape(x_train, (n, 1)) for i=1:p], [y_train[:,i:i] for i=1:p], kernel=k)
gp."mixed_noise"."Gaussian_noise_0".variance = 0.001
gp."mixed_noise"."Gaussian_noise_1".variance = 0.001
gp."mixed_noise"."Gaussian_noise_2".variance = 0.001
gp."mixed_noise"."Gaussian_noise_3".variance = 0.001
gp."mixed_noise"."Gaussian_noise_4".variance = 0.001

println("Before => MLL_cov(train): ", gp.objective_function())
for i=1:p
    println("Before => MSE_$(i)(test): ", mse(gp.predict(hcat(x_test, (i-1)*ones(Int64, n)), Y_metadata=Dict("output_index" => [i-1 for j=1:n]))[1], y_test[:,i]))
end

gp.optimize()
println("---")

println("After => MLL_cov(train): ", gp.objective_function())
for i=1:p
    println("After => MSE_$(i)(test): ", mse(gp.predict(hcat(x_test, (i-1)*ones(Int64, n)), Y_metadata=Dict("output_index" => [i-1 for j=1:n]))[1], y_test[:,i]))
end

GPy
---
Before => MLL_cov(train): 121438.73472877713
Before => MSE_1(test): 0.4417629761225589
Before => MSE_2(test): 0.5123235315954173
Before => MSE_3(test): 0.4350687663462204
Before => MSE_4(test): 0.46757976322368217
Before => MSE_5(test): 0.45750958963607485
---
After => MLL_cov(train): -9.533891613049974
After => MSE_1(test): 0.0323054442034474
After => MSE_2(test): 0.044964068328791046
After => MSE_3(test): 0.03378538619952279
After => MSE_4(test): 0.0484520468778252
After => MSE_5(test): 0.031411676773043615


### 3.3. Fixed rank = 3 coregionalization matrix

In [46]:
println("GPForecasting")
println("-------------")
U, S, V = svd(cov(y_train))
H = U * Diagonal(sqrt.(S))[:,1:3]
gp = GPForecasting.GP(OLMMKernel(3, p, 0.0, [0.001 for i=1:3], H, [(EQ() ▷ 1.0) for i=1:3]))
before = condition(gp, x_train, y_train)
b = before.m(x_test)
println("Before => MLL_cov(train): ", -logpdf(gp, x_train, y_train))
for i=1:p
    println("Before => MSE_$(i)(test): ", mse(b[:,i], y_test[:,i]))
end

gp = learn(gp, x_train, y_train, objective, trace=false)
println("-------------")

after = condition(gp, x_train, y_train)
a = after.m(x_test)
println("After => MLL_cov(train): ", -logpdf(gp, x_train, y_train))
for i=1:p
    println("After => MSE_$(i)(test): ", mse(a[:,i], y_test[:,i]))
end

GPForecasting
-------------
Before => MLL_cov(train): 717607.8919663227
Before => MSE_1(test): 0.4397352304458172
Before => MSE_2(test): 0.5166727863649464
Before => MSE_3(test): 0.43872550877010497
Before => MSE_4(test): 0.4573095643804965
Before => MSE_5(test): 0.4559606375143938
-------------
After => MLL_cov(train): -167.2241981236382
After => MSE_1(test): 0.044906944786545194
After => MSE_2(test): 0.060189020445752854
After => MSE_3(test): 0.045324188900885104
After => MSE_4(test): 0.07436374601653983
After => MSE_5(test): 0.04701795422700438


In [47]:
println("GPy")
println("---")
U, S, V = svd(cov(y_train))
H = U * Diagonal(sqrt.(S))[:,1:3]
kappa = zeros(5)
K = GPy.kern.RBF(input_dim=1, variance=1.0, lengthscale=1.0)
K."variance".fix()
k = GPy.util.multioutput.ICM(input_dim=1, num_outputs=5, kernel=K, W_rank=3, W=H, kappa=kappa)
k."B"."W".fix()
k."B"."kappa".fix()
gp = GPy.models.GPCoregionalizedRegression([reshape(x_train, (n, 1)) for i=1:p], [y_train[:,i:i] for i=1:p], kernel=k)
gp."mixed_noise"."Gaussian_noise_0".variance = 0.001
gp."mixed_noise"."Gaussian_noise_1".variance = 0.001
gp."mixed_noise"."Gaussian_noise_2".variance = 0.001
gp."mixed_noise"."Gaussian_noise_3".variance = 0.001
gp."mixed_noise"."Gaussian_noise_4".variance = 0.001

println("Before => MLL_cov(train): ", gp.objective_function())
for i=1:p
    println("Before => MSE_$(i)(test): ", mse(gp.predict(hcat(x_test, (i-1)*ones(Int64, n)), Y_metadata=Dict("output_index" => [i-1 for j=1:n]))[1], y_test[:,i]))
end

gp.optimize()
println("---")

println("After => MLL_cov(train): ", gp.objective_function())
for i=1:p
    println("After => MSE_$(i)(test): ", mse(gp.predict(hcat(x_test, (i-1)*ones(Int64, n)), Y_metadata=Dict("output_index" => [i-1 for j=1:n]))[1], y_test[:,i]))
end

GPy
---
Before => MLL_cov(train): 121510.65665195147
Before => MSE_1(test): 0.4418281589870574
Before => MSE_2(test): 0.5125821765244223
Before => MSE_3(test): 0.43557414993598376
Before => MSE_4(test): 0.4682367380874273
Before => MSE_5(test): 0.4578379151938163
---
After => MLL_cov(train): -12.118713101147648
After => MSE_1(test): 0.0329529227952368
After => MSE_2(test): 0.047290871358758944
After => MSE_3(test): 0.03543107436908517
After => MSE_4(test): 0.051982404684390096
After => MSE_5(test): 0.03255413227247944
